In [1]:
import sys
sys.path.append("..")

import deeptrack as dt
#import graphtrack as gt
from deeptrack.models.gnns.generators import GraphGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
import logging


logging.disable(logging.WARNING)



c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\deeptrack\backend\_config.py:11: UserWarning: cupy not installed. GPU-accelerated simulations will not be possible
  warnings.warn(
c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\deeptrack\backend\_config.py:25: UserWarning: cupy not installed, CPU acceleration not enabled
  warnings.warn("cupy not installed, CPU acceleration not enabled")


In [2]:
import os
import glob

import re

import tqdm
import untangle
from graphtrack import graphs

_default_properties = {"intensity": 70.0, "radius": 3}

def NodeExtractor(
    paths=None,
    properties: dict = _default_properties,
    extract_solution=False,
    **kwargs,
):
    def to_frame(xml):
        particles = xml.root.GlobalHeterogenousInfo.particle

        detection_list = []
        for p in range(len(particles)):
            detections = particles[p].detection
            for d in range(len(detections)):
                detection_att = {
                    "frame": int(detections[d]["frame"]),
                    "centroid_x": float(detections[d]["x"]),
                    "centroid_y": float(detections[d]["y"]),
                    "intensity": float(detections[d]["intensity"]),
                    "radius": float(detections[d]["radius"]),
                    "label": int(p + 1),
                }
                if extract_solution:
                    detection_att["solution"] = float(
                        detections[d]["solution"]
                    )

                detection_list.append(detection_att)
        df = (
            pd.DataFrame.from_dict(detection_list)
            .sort_values(by=["frame"])
            .reset_index(drop=True)
        )
        df["solution"] = 0.0
        return df, np.array([0.0])

    _properties = {
        "label": 1,
        "centroid": np.array([128.0, 128.0]).astype(np.float32),
    }
    _properties.update(properties)

    dfs, global_property = [], []
    for batch, path in tqdm.tqdm(
        enumerate(paths), total=len(paths), desc="Loading xmls"
    ):
        df, global_prop = to_frame(untangle.parse(path))

        # Normalize features
        df.loc[:, df.columns.str.contains("centroid")] = np.round(
            df.loc[:, df.columns.str.contains("centroid")]
            / _properties["centroid"],
            3,
        )
        df["intensity"] = np.round(
            df["intensity"] / _properties["intensity"], 3
        )
        df["radius"] = np.round(df["radius"] / _properties["radius"], 3)

        # Append solution
        if not ("solution" in df.columns):
            df["solution"] = 0.0

        # Append set
        df["set"] = batch
        dfs.append(df)
        global_property.append(global_prop)

    dfs = pd.concat(dfs)
    # dfs = dfs.clip(lower=0)

    return dfs, list(_properties.keys()), global_property


_path_to_xml = os.path.join(
    ".", "xml-generators", "xml_data", "{_type}", "{mode}", "*.xml"
)


def LoadGraphXml(_type="mixed_alpha", mode="training", **kwargs):
    PATH_TO_DATASET = glob.glob(_path_to_xml.format(_type=_type, mode=mode))
    nodesdf, props, global_property = NodeExtractor(PATH_TO_DATASET, **kwargs)

   
    return nodesdf

c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\graphtrack\deeptrack\backend\_config.py:11: UserWarning: cupy not installed. GPU-accelerated simulations will not be possible
  warnings.warn(
c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\graphtrack\deeptrack\backend\_config.py:25: UserWarning: cupy not installed, CPU acceleration not enabled
  warnings.warn("cupy not installed, CPU acceleration not enabled")


In [3]:
nodesdf = LoadGraphXml(_type="mixed_alpha_nodes", mode="training")

Loading xmls: 100%|██████████| 4/4 [00:00<00:00, 29.29it/s]


In [4]:
nodesdf.head(20)

,frame,centroid_x,centroid_y,intensity,radius,label,solution,set
0,0,0.450,0.447,1.993,0.757,1,0.0,0
1,0,0.636,0.036,1.933,0.626,27,0.0,0
2,0,0.602,0.409,2.060,0.841,28,0.0,0
3,0,0.322,0.026,1.997,0.533,52,0.0,0
4,0,0.179,0.331,2.009,0.704,29,0.0,0
5,0,0.666,0.414,1.893,0.538,31,0.0,0
6,0,0.204,0.489,2.065,0.599,32,0.0,0
7,0,0.122,0.528,1.955,0.551,4,0.0,0
8,0,0.725,0.561,1.987,0.781,33,0.0,0
9,0,0.339,0.336,1.919,0.656,34,0.0,0


In [5]:
# Seach radius for the graph edges
radius = 0.20

# Output type
_OUTPUT_TYPE = "edges"

variables = dt.DummyFeature(
    radius=radius,
    output_type=_OUTPUT_TYPE,
    nofframes=3
)

In [37]:
model = dt.models.gnns.MAGIK(
    dense_layer_dimensions=(32, 64, 96),      # number of features in each dense encoder layer
    base_layer_dimensions=(96, 96),    # Latent dimension throughout the message passing layers
    number_of_node_features=4,             # Number of node features in the graphs
    number_of_edge_features=1,             # Number of edge features in the graphs
    number_of_edge_outputs=1,              # Number of predicted features
    edge_output_activation="sigmoid",      # Activation function for the output layer
    output_type=_OUTPUT_TYPE,              # Output type. Either "edges", "nodes", or "graph"
)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'binary_crossentropy',
    metrics=['accuracy'],
)

model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           [(None, None, 4)]    0                                            
__________________________________________________________________________________________________
node_ide1 (Dense)               (None, None, 33)     165         input_33[0][0]                   
__________________________________________________________________________________________________
lambda (Lambda)                 multiple             0           node_ide1[0][0]                  
                                                                 edge_ide1[0][0]                  
                                                                 node_ide2[0][0]                  
                                                                 edge_ide2[0][0]            

In [21]:
_LOAD_MODEL = False

if _LOAD_MODEL:
    print("Loading model...")
    model.load_weights("datasets/BFC2DLMuSCTra/MAGIK.h5")
else:
    generator = GraphGenerator(
        nodesdf=nodesdf,
        properties=["intensity", "radius", "centroid"],
        #parenthood=parenthood,
        min_data_size=100,
        max_data_size=101,
        batch_size=8,
        **variables.properties()
    )
    
    

Creating graph edges...


100%|██████████| 4/4 [00:09<00:00,  2.31s/it]


In [32]:
np.shape(generator[1][0][0])

(8, 535, 4)

In [8]:
with generator:
    data, labels = generator[0]

c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\deeptrack2\deeptrack\image.py:289: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(self.to_numpy()._value)


Generating 512 / 511 samples before starting training


In [10]:
data[0]

array([[[ 0.7072249 ,  0.43906929,  2.03206803,  0.58096944],
        [ 0.15234836,  0.47138062,  1.99207124,  0.51239243],
        [ 0.72570631,  0.28175293,  1.94922277,  0.83503766],
        ...,
        [ 0.36091704,  0.46871493,  2.09018914,  0.72584987],
        [ 0.77010331,  0.67483041,  1.89581341,  0.63242922],
        [ 0.49510866,  0.81465511,  2.02832259,  0.71947828]],

       [[ 0.764792  ,  0.41810495,  1.99152183,  0.76533821],
        [ 0.14757924,  0.57703389,  2.00778593,  0.71393201],
        [ 0.26809256,  0.59835384,  1.88261586,  0.58228696],
        ...,
        [ 0.69084753,  0.4909144 ,  1.96899826,  0.79322906],
        [ 0.27900056,  0.2055969 ,  1.88505818,  0.74266429],
        [ 0.63827976,  0.0385267 ,  1.97335611,  0.71086788]],

       [[ 0.50425589,  0.30286437,  1.93667809,  0.64688031],
        [ 0.19709227,  0.40554596,  1.88859405,  0.5660638 ],
        [ 0.42564916,  0.21338262,  1.9191428 ,  0.56122456],
        ...,
        [ 0.26103685,  0.70

In [38]:
with generator:
        model.fit(generator, epochs=10)

Generating 101 / 100 samples before starting training
Epoch 1/10


InvalidArgumentError:  indices[0,3159,1] = 601 is not in [0, 589)
	 [[node model_6/graph_block_0/GatherV2 (defined at c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\deeptrack2\deeptrack\models\gnns\layers.py:83) ]] [Op:__inference_train_function_79616]

Errors may have originated from an input operation.
Input Source operations connected to node model_6/graph_block_0/GatherV2:
 model_6/layer_normalization_108/add (defined at c:\Users\santh\Desktop\Python Projects\Masters Thesis\Multi_particleTracking\GraphTrack\deeptrack2\deeptrack\models\utils.py:231)

Function call stack:
train_function


In [ ]:
model.load_weights("./gnn1")

## Testing experimental data

In [ ]:
exp_data = pd.read_csv('df.csv') 
exp_data

In [ ]:
exp_data.drop(exp_data[exp_data["frame"]>10].index, inplace = True)

In [ ]:
pred, gt, scores, graph = dt.models.gnns.get_predictions(
    exp_data, ["intensity", "centroid"], model, variables
)

In [ ]:
edges_df, nodes, _ = dt.models.gnns.df_from_results(pred, gt, scores, graph)

In [ ]:
pred_trajs = dt.models.gnns.get_traj(edges_df, th = 8)

In [ ]:
for (t,c) in pred_trajs:
    x = exp_data.take(t)["centroid_x"].values*530
    y = exp_data.take(t)["centroid_y"].values*100

    plt.scatter(x, y, s=1)
    plt.title("Predicted trajectories")
    